In [6]:
### img -> tf record withot pickling

import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os

from keras.utils import to_categorical
from sklearn import preprocessing

from PIL import Image
import itertools
from tqdm import tqdm

import numpy as np
import skimage.io as io
import tensorflow as tf

from tqdm import tqdm
from dltk.io.preprocessing import *
import scipy.ndimage


import pickle

root = 'D:\data\segmentiation'
ct_set = os.path.join(root,'ct_train_test/ct_train/')
mr_set = os.path.join(root,'mr_train_test/mr_train/')
filenames = os.listdir(ct_set)



In [7]:
def pickle_iter(path):
    f = open(path, 'rb')
    unpickler = pickle.Unpickler(f)
    try:
        for i in range(9999999999):
            yield unpickler.load()
    except:
        f.close()
        print('pickle generator created')

        
def pad3d(array):
    height = array.shape[0]
    depth = array.shape[2]
    
    if (height - depth) % 2 :
        pad_front = int((height + 1 - depth) / 2)
        pad_back = int((height - 1 - depth) / 2)
    else:
        pad_front = pad_back = int((height - depth) / 2)
    
    npad = ((0,0),(0,0),(pad_front,pad_back))
    array_padding = np.pad(array, npad, 'constant', constant_values=(0))
    array_padding[array_padding<0] = 0
    
    return array_padding

def pad3d(array):
    x0 = array.shape[0]
    x2 = array.shape[2]    
    if x0 > x2:
        height = x0
        depth = x2
    elif x2 > x0:
        height = x2
        depth = x0
    
    if (height - depth) % 2 :
        pad_front = int((height + 1 - depth) / 2)
        pad_back = int((height - 1 - depth) / 2)
    else:
        pad_front = pad_back = int((height - depth) / 2)
    
    if x0 > x2:    
        npad = ((0,0),(0,0),(pad_front,pad_back))
    elif x2 > x0:
        npad = ((pad_front,pad_back),(0,0),(0,0))
        
    array_padding = np.pad(array, npad, 'constant', constant_values=(0))
    array_padding[array_padding<0] = 0
    
    return array_padding

def image_preprocess(image, new_size, mask=False):
    assert np.sum(image.shape==image.shape[0])!=3    
    
    ratio = new_size / image.shape[0]
    
    image = scipy.ndimage.zoom(image, zoom=ratio, order=0)

    if mask:
        channel = 7 + 1 #background
        image = image.reshape(-1)
        
        
        image = label_encoder.fit_transform(image)
        
        
        
        
        print("annotation shape",image.shape)
        print("unique value", np.unique(image))
        
        
        image = to_categorical(image, class_num)
        print("processed mask shape",image.shape)

        
    else:
        channel = 1
    # reshape to raw shape
    image = image.reshape((new_size,)*3 + (channel,))
    print("Reshaped shape",image.shape)
    
    return image

def nii_loader(path,file_name):
    file_path =  os.path.join(path,file_name)
    fn = os.listdir(file_path)
    image = (nib.load(file_path + '/' + fn[0]))
    return image

def axis_transform(image):
    idx = 0
    if np.sum(image[2,:,int(image.shape[-1]/2)]) == 0:
        image = image.T
        image = np.flip(image,axis=0)
        idx += 1
    return image, idx

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def write_image_annotation_pairs_to_tfrecord_from_gen(generator, tfrecords_filename, test=False):
    """Writes given image/annotation pairs to the tfrecords file.
    The function reads each image/anno'tation pair given filenames
    of image and respective annotation and writes it to the tfrecord
    file.
    Parameters
    ----------
    filename_pairs : array of tuples (img_filepath, annotation_filepath)
        Array of tuples of image/annotation filenames
    tfrecords_filename : string
        Tfrecords filename to write the image/annotation pairs
    """
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
    writer = tf.python_io.TFRecordWriter(tfrecords_filename,options=options)
    
    print("Start writing tfrecord")


    #for img_path, annotation_path in filename_pairs:

    for i, record in enumerate(generator):
        print("Processing image",i)

        #img = np.array(Image.open(img_path))

        #annotation = np.array(Image.open(annotation_path))

        # Unomment this one when working with surgical data
        # annotation = annotation[:, :, 0]

        # The reason to store image sizes was demonstrated
        # in the previous example -- we have to know sizes
        # of images to later read raw serialized string,
        
        # convert to 1d array and convert to respective
        # shape that image used to have.
        #height = img.shape[0] 
        

        if test :
            img = record
        else :
            img = record[0]
            
        height = img.shape[0] 

        #width = img.shape[1]
        width = img.shape[1] 

        #add depth 
        depth = img.shape[2]
        print("Img shape when loaded",img.shape)

        #print(depth)
        #print(img[0].shape) #288,288,140 

       #img_raw = img.tostring()
        img_raw=img.tostring()

        ## 여기서 에러
        print("img shape: {}, img_raw shape: {}".format(img.shape,len(img_raw)))        

        if test == False :
            annotation = record[1]
            annotation_raw = annotation.tostring()
            print("annotation_ shape: {}, annotation_raw shape: {}".format(annotation.shape,len(annotation_raw)))

            example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(height),
                'width': _int64_feature(width),
                'depth': _int64_feature(depth), 
                'image_raw': _bytes_feature(img_raw),
                'mask_raw': _bytes_feature(annotation_raw)}))
        else:
            example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(height),
                'width': _int64_feature(width),
                'depth': _int64_feature(depth), 
                'image_raw': _bytes_feature(img_raw)
                }))
            

        writer.write(example.SerializeToString())

    writer.close()
    
def write_image_annotation_pairs_to_tfrecord_from_listitr(pickle_itr_list, tfrecords_filename):
    """Writes given image/annotation pairs to the tfrecords file.
    The function reads each image/anno'tation pair given filenames
    of image and respective annotation and writes it to the tfrecord
    file.
    Parameters
    ----------
    filename_pairs : array of tuples (img_filepath, annotation_filepath)
        Array of tuples of image/annotation filenames
    tfrecords_filename : string
        Tfrecords filename to write the image/annotation pairs
    """
    
    print("Start writing tfrecord")
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
    writer = tf.python_io.TFRecordWriter(tfrecords_filename,options=options)

    #for img_path, annotation_path in filename_pairs:
    
    
    for pickle_itr in pickle_itr_list:

        for i,record in enumerate(pickle_itr):
            print("Processing image",i)
            print(record.keys())

            img = record['image']
            annotation = record['label']

            #img = np.array(Image.open(img_path))

            #annotation = np.array(Image.open(annotation_path))

            # Unomment this one when working with surgical data
            # annotation = annotation[:, :, 0]

            # The reason to store image sizes was demonstrated
            # in the previous example -- we have to know sizes
            # of images to later read raw serialized string,
            # convert to 1d array and convert to respective
            # shape that image used to have.
            #height = img.shape[0] 

            print("Img shape when loaded",img.shape)
            ##### img = img.reshape()
            
            
            height = img.shape[0] 

            #width = img.shape[1]
            width = img.shape[1] 


            #add depth 
            depth = img.shape[2]
            #print(depth)
            #print(img[0].shape) #288,288,140 


            #img_raw = img.tostring()
            img_raw=img.tostring()

            ## 여기서 에러
            print("img shape: {}, img_raw shape: {}".format(img.shape,len(img_raw)))

            ## 여기서 에러
            annotation_raw = annotation.tostring()
            print("annotation_ shape: {}, annotation_raw shape: {}".format(annotation.shape,len(annotation_raw)))

            #print(annotation[1].shape)
            #print(annotation[2].shape)


            #print(annotation_raw)
            #annotation_raw = annotation.tostring()

            example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(height),
                'width': _int64_feature(width),
                'depth': _int64_feature(depth), 
                'image_raw': _bytes_feature(img_raw),
                'mask_raw': _bytes_feature(annotation_raw)}))

            writer.write(example.SerializeToString())

    writer.close()
    
def read_image_annotation_pairs_from_tfrecord_itr_from_compressed(tfrecords_filename):
    """Return image/annotation pairs from the tfrecords file.
    The function reads the tfrecords file and returns image
    and respective annotation matrices pairs.
    Parameters
    ----------
    tfrecords_filename : string
        filename of .tfrecords file to read from
    
    Returns
    -------
    image_annotation_pairs : array of tuples (img, annotation)
        The image and annotation that were read from the file
    """
    image_annotation_pairs = []
    
    ## option...
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)

    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename,options=options)

    for string_record in record_iterator:

        example = tf.train.Example()
        example.ParseFromString(string_record)

        height = int(example.features.feature['height']
                                     .int64_list
                                     .value[0])

        width = int(example.features.feature['width']
                                    .int64_list
                                    .value[0])
        
        depth = int(example.features.feature['depth']
                                    .int64_list
                                    .value[0])

        img_string = (example.features.feature['image_raw']
                                      .bytes_list
                                      .value[0])
        

        annotation_string = (example.features.feature['mask_raw']
                                    .bytes_list
                                    .value[0])
        
        # img_1d = np.fromstring(img_string, dtype=np.uint8)
        img_1d = np.fromstring(img_string)
                               
        print("img_string shape",img_1d.shape)
        print("image :  {}, annotation length : {}".format(len(img_string),len(annotation_string)))

        #        img = img_1d.reshape((height, width, depth, -1))

        img = img_1d.reshape((height, width, depth,1))

        annotation_1d = np.fromstring(annotation_string, dtype=np.float32)
#        annotation_1d = np.fromstring(annotation_string)

        # Annotations don't have depth (3rd dimension)
        # TODO: check if it works for other datasets
        #  annotation = annotation_1d.reshape((height, width, depth,-1))

        annotation = annotation_1d.reshape((height, width, depth, 8))
        
        yield img, annotation

In [8]:
def pre_process_tfrecord_write_itr(path,itr,new_size, test=False):
    for record in tqdm(itr):
        file_name = record['name']
        img_fname = record['image']
        
        print(path,img_fname)
        
        ## nii 데이터 불러오기
        img = nii_loader(path,img_fname)

        print("loaded Image Shape",img.get_data().shape)
        
        ## 이미지 padding + resizing
        img = pad3d(img.get_data())
        img = image_preprocess(img,new_size=new_size)
        
        ## 레이블 padding + resizing
        if test :
            print("Image shape before yielding",img.shape)
            yield img

        else:
            label_fname = record['label']
            lab = nii_loader(path, label_fname)
            lab = pad3d(lab.get_data())
            lab = image_preprocess(lab,new_size=new_size,mask=True)

            print("Mask shape when preproceesing ends",lab.shape)
            
            yield img, lab
        

        


## CT train Data TF record Encoding

In [9]:
ct_set = os.path.join(root,'ct_train_test/ct_train/')
ct_set_test = os.path.join(root,'ct_train_test/ct_test/')
mr_set = os.path.join(root,'mr_train_test/mr_train/')
mr_set_test =  os.path.join(root,'ct_train_test/mr_test/')

## 여기 바꾸면서 ct, mr 파일 만들기
temp_path = ct_set

name_list = []
filenames = os.listdir(temp_path)

for i in filenames:
    if i[:13] not in name_list:
        name_list.append(i[:13])
        
total_list = []
for i in name_list:
    temp_name = i
    temp_image = i+'_image.nii'
    temp_label = i+'_label.nii'
    total_list.append({'name' : temp_name,'image':temp_image,'label':temp_label})

label_encoder = preprocessing.LabelEncoder()
class_num=7+1
print("starting gen")


## 전처리 Generator
preprocess_gen = pre_process_tfrecord_write_itr(temp_path,total_list,new_size=256)


out_file = temp_path.split('/')[-2] + '_tfrecord'
tfrecords_filename =os.path.join(root, out_file)

## Tf Record 형식으로 저장
write_image_annotation_pairs_to_tfrecord_from_gen(preprocess_gen,tfrecords_filename)

starting gen
Start writing tfrecord


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

D:\data\segmentiation\ct_train_test/ct_train/ ct_train_1001_image.nii
loaded Image Shape (512, 512, 363)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
Processing image 0
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


  5%|████▏                                                                              | 1/20 [00:27<08:38, 27.28s/it]

D:\data\segmentiation\ct_train_test/ct_train/ ct_train_1002_image.nii
loaded Image Shape (512, 512, 239)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
Processing image 1
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [06:49<00:45, 22.90s/it]

D:\data\segmentiation\ct_train_test/ct_train/ ct_train_1019_image.nii
loaded Image Shape (512, 512, 297)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
Processing image 18
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [07:13<00:23, 23.30s/it]

D:\data\segmentiation\ct_train_test/ct_train/ ct_train_1020_image.nii
loaded Image Shape (512, 512, 363)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
Processing image 19
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [07:41<00:00, 24.56s/it]


## Decoding test

In [10]:
tfr_itr = read_image_annotation_pairs_from_tfrecord_itr_from_compressed(tfrecords_filename)

In [11]:

cnt = 0 
for i,j in tfr_itr:
    decoded_image = i
    decoded_annotation = j
    if cnt == 1:
        break
    cnt +=1 
    
preprocess_gen = pre_process_tfrecord_write_itr(temp_path,total_list,new_size=256)

cnt = 0

for a,b in preprocess_gen:
    image = a
    annotation = b
    
    if cnt == 1:
        break
    cnt += 1

c:\program files\anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:329: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


img_string shape (16777216,)
image :  134217728, annotation length : 536870912


c:\program files\anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:338: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


img_string shape (16777216,)
image :  134217728, annotation length : 536870912


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

D:\data\segmentiation\ct_train_test/ct_train/ ct_train_1001_image.nii
loaded Image Shape (512, 512, 363)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)


  5%|████▏                                                                              | 1/20 [00:12<04:03, 12.81s/it]

D:\data\segmentiation\ct_train_test/ct_train/ ct_train_1002_image.nii
loaded Image Shape (512, 512, 239)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)


In [12]:
decoded_image ==  image
decoded_annotation == annotation


array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  T

In [13]:
decoded_image.shape

(256, 256, 256, 1)

## Ct test TF record Encoding


In [14]:
ct_set = os.path.join(root,'ct_train_test/ct_train/')
ct_set_test = os.path.join(root,'ct_train_test/ct_test/')
mr_set = os.path.join(root,'mr_train_test/mr_train/')
mr_set_test =  os.path.join(root,'ct_train_test/mr_test/')

## 여기 바꾸면서 ct, mr 파일 만들기
temp_path = ct_set_test

name_list = []
filenames = os.listdir(temp_path)

for i in filenames:
    if i[:12] not in name_list:
        name_list.append(i[:12])
        
total_list = []
for i in name_list:
    temp_name = i
    temp_image = i+'_image.nii'
    temp_label = i+'_label.nii'
    total_list.append({'name' : temp_name,'image':temp_image,'label':temp_label})

# label_encoder = preprocessing.LabelEncoder()
# class_num=7+1
print("starting gen")

#0# 전처리 Generator
preprocess_gen = pre_process_tfrecord_write_itr(temp_path,total_list,new_size=256,test=True)

out_file = temp_path.split('/')[-2] + '_tfrecord'
tfrecords_filename =os.path.join(root, out_file)

## Tf Record 형식으로 저장
write_image_annotation_pairs_to_tfrecord_from_gen(preprocess_gen,tfrecords_filename, test=True)

starting gen
Start writing tfrecord



  0%|                                                                                           | 0/40 [00:00<?, ?it/s]

D:\data\segmentiation\ct_train_test/ct_test/ ct_test_2001_image.nii
loaded Image Shape (512, 512, 224)
Reshaped shape (256, 256, 256, 1)
Image shape before yielding (256, 256, 256, 1)
Processing image 0
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728



  2%|██                                                                                 | 1/40 [00:09<06:19,  9.73s/it]

D:\data\segmentiation\ct_train_test/ct_test/ ct_test_2002_image.nii
loaded Image Shape (512, 512, 200)
Reshaped shape (256, 256, 256, 1)
Image shape before yielding (256, 256, 256, 1)
Processing image 1
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728


...


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 38/40 [06:45<00:19,  9.97s/it]

D:\data\segmentiation\ct_train_test/ct_test/ ct_test_2039_image.nii
loaded Image Shape (512, 512, 177)
Reshaped shape (256, 256, 256, 1)
Image shape before yielding (256, 256, 256, 1)
Processing image 38
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728



 98%|███████████████████████████████████████████████████████████████████████████████▉  | 39/40 [06:54<00:09,  9.71s/it]

D:\data\segmentiation\ct_train_test/ct_test/ ct_test_2040_image.nii
loaded Image Shape (512, 512, 283)
Reshaped shape (256, 256, 256, 1)
Image shape before yielding (256, 256, 256, 1)
Processing image 39
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728



100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [07:06<00:00, 10.39s/it]


### MR train TF record Encoding
* need to fix ceratin mr data value (mr은 값을 하나 바꿔줘야됨)
* change axis (CT에서 의미하는 축과  MR 의미하는 축이 다른 경우가 있음 통일할 필요) 
* https://github.com/bllfpc/TTokDak/blob/master/ipynb/image%20padding.ipynb

In [15]:
## utils

def image_preprocess(image, new_size, mask=False):
    assert np.sum(image.shape==image.shape[0])!=3    
    
    ratio = new_size / image.shape[0]
    
    image = scipy.ndimage.zoom(image, zoom=ratio, order=0)

    if mask:
        channel = 7 + 1 #background
        image = image.reshape(-1)
        
        
        image = label_encoder.fit_transform(image)
        
        
        
        
        print("annotation shape",image.shape)
        print("unique value", np.unique(image))
        
        
        image = to_categorical(image, class_num)
        print("processed mask shape",image.shape)

        
    else:
        channel = 1
    # reshape to raw shape
    image = image.reshape((new_size,)*3 + (channel,))
    print("Reshaped shape",image.shape)
    
    return image

def pad3d(array):
    
    print("Starting Padding Image shape",array.shape)
    
    x0 = array.shape[0]
    x2 = array.shape[2]    
    if x0 > x2:
        height = x0
        depth = x2
    elif x2 > x0:
        height = x2
        depth = x0
    
    if (height - depth) % 2 :
        pad_front = int((height + 1 - depth) / 2)
        pad_back = int((height - 1 - depth) / 2)
    else:
        pad_front = pad_back = int((height - depth) / 2)
    
    if x0 > x2:    
        npad = ((0,0),(0,0),(pad_front,pad_back))
    elif x2 > x0:
        npad = ((pad_front,pad_back),(0,0),(0,0))
        
    array_padding = np.pad(array, npad, 'constant', constant_values=(0))
    array_padding[array_padding<0] = 0
    
    return array_padding


def axis_transform(image):
    idx = 0
    if np.sum(image[2,:,int(image.shape[-1]/2)]) == 0:
        image = image.T
        image = np.flip(image,axis=0)
        idx += 1
    return image, idx

def pre_process_tfrecord_write_itr_mr(path,itr,new_size, test=False):
    for record in tqdm(itr):
        file_name = record['name']
        img_fname = record['image']
        
        print(path,img_fname)
        
        ## nii 데이터 불러오기
        img = nii_loader(path,img_fname)

        print("loaded Image Shape",img.get_data().shape)
        
        ## 이미지 padding + resizing
        img, idx = axis_transform(img.get_data())
        print("transformed image shape",img.shape)
        img = pad3d(img)
        img = image_preprocess(img,new_size=new_size)
        
        ## 레이블 padding + resizing
        if test :
            print("Image shape before yielding",img.shape)
            yield img

        else:
            label_fname = record['label']
            lab = nii_loader(path, label_fname)
            lab = lab.get_data()
            
            if label_fname == 'mr_train_1010_label.nii':
                lab[lab==421]= 420
                print("Outlier detected")
            
            if idx :
                lab, _ = axis_transform(lab)
                
            lab = pad3d(lab)
            lab = image_preprocess(lab,new_size=new_size,mask=True)

            print("Mask shape when preproceesing ends",lab.shape)
            
            yield img, lab

In [16]:
mr_set = os.path.join(root,'mr_train_test/mr_train/')
mr_set_test =  os.path.join(root,'mr_train_test/mr_test/')

## 여기 바꾸면서 ct, mr 파일 만들기
temp_path = mr_set

name_list = []
filenames = os.listdir(temp_path)

for i in filenames:
    if i[:13] not in name_list:
        name_list.append(i[:13])
        
total_list = []
for i in name_list:
    temp_name = i
    temp_image = i+'_image.nii'
    temp_label = i+'_label.nii'
    total_list.append({'name' : temp_name,'image':temp_image,'label':temp_label})

    
# Label Encoder 선언
label_encoder = preprocessing.LabelEncoder()
class_num=7+1
print("starting gen")
    
    
## 전처리 Generator
preprocess_gen = pre_process_tfrecord_write_itr_mr(temp_path,total_list,new_size=256,test =False)


out_file = temp_path.split('/')[-2] + '_tfrecord'
tfrecords_filename =os.path.join(root, out_file)

## Tf Record 형식으로 저장
write_image_annotation_pairs_to_tfrecord_from_gen(preprocess_gen,tfrecords_filename, test=False)


starting gen
Start writing tfrecord



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

D:\data\segmentiation\mr_train_test/mr_train/ mr_train_1001_image.nii
loaded Image Shape (512, 512, 160)
transformed image shape (512, 512, 160)
Starting Padding Image shape (512, 512, 160)
Reshaped shape (256, 256, 256, 1)
Starting Padding Image shape (512, 512, 160)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
Processing image 0
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 33554432
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912



  5%|████▏                                                                              | 1/20 [00:12<03:52, 12.23s/it]

D:\data\segmentiation\mr_train_test/mr_train/ mr_train_1002_image.nii
loaded Image Shape (512, 512, 128)
transformed image shape (512, 512, 128)
Starting Padding Image shape (512, 512, 128)
Reshaped shape (256, 256, 256, 1)
Starting Padding Image shape (512, 512, 128)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
Processing image 1
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 33554432
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


...


 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [03:30<00:23, 11.69s/it]

D:\data\segmentiation\mr_train_test/mr_train/ mr_train_1019_image.nii
loaded Image Shape (288, 288, 135)
transformed image shape (135, 288, 288)
Starting Padding Image shape (135, 288, 288)
Reshaped shape (256, 256, 256, 1)
Starting Padding Image shape (135, 288, 288)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
Processing image 18
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 33554432
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912



 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [03:41<00:11, 11.60s/it]

D:\data\segmentiation\mr_train_test/mr_train/ mr_train_1020_image.nii
loaded Image Shape (288, 288, 135)
transformed image shape (135, 288, 288)
Starting Padding Image shape (135, 288, 288)
Reshaped shape (256, 256, 256, 1)
Starting Padding Image shape (135, 288, 288)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
Processing image 19
Img shape when loaded (256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 33554432
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:53<00:00, 11.52s/it]


## For Memory Error (Optional)

In [12]:
## For Low memory Setting
## preprocessing -> pickling -> tfrecord

outpath = r"D:\segmentiation\scipy_pickle_test0.pkl"
memory_error_list = []
cnt = 0

label_encoder = preprocessing.LabelEncoder()
class_num=7+1
## Making padded image

todolist = [i for i in total_list]



with open(outpath,"wb") as f:
    pickler = pickle.Pickler(f)
    for  record in tqdm(todolist):


        try :

            file_name = record['name']
            img_fname = record['image']
            label_fname = record['label']



            # nii load // 이 결과가가 원래 결과랑 같은 지 확인 하자
            img = nii_loader(ct_set,img_fname)
            lab = nii_loader(ct_set, label_fname)


            img = pad3d(img.get_data())
            img = image_preprocess(img,new_size=256)

            lab = pad3d(lab.get_data())
            lab = image_preprocess(lab,new_size=256,mask=True)



            print("Start MASKING"+"====="*10)

            List = {'filename':file_name,'image': img, 'label':  lab}
            pickler.dump(List)

            del file_name, img_fname, label_fname, img, lab
            
            print("Dumping",cnt)
            cnt += 1

        except MemoryError:
            print("Memory Error")
            todolist = total_list[cnt:]
            break

outpath = outpath[:-5] + str(int(outpath[-5]) + 1) + str(".pkl")



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================




  5%|████▏                                                                              | 1/20 [00:35<11:17, 35.67s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================




 10%|████████▎                                                                          | 2/20 [01:11<10:42, 35.71s/it]

...



 35%|█████████████████████████████                                                      | 7/20 [04:14<07:55, 36.57s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================




 40%|█████████████████████████████████▏                                                 | 8/20 [04:48<07:09, 35.83s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================




 45%|█████████████████████████████████████▎                                             | 9/20 [05:28<06:48, 37.14s/it]

In [24]:
with open(outpath,"wb") as f:
    pickler = pickle.Pickler(f)
    for  record in tqdm(todolist):


        try :

            file_name = record['name']
            img_fname = record['image']
            label_fname = record['label']



            # nii load // 이 결과가가 원래 결과랑 같은 지 확인 하자
            img = nii_loader(ct_set,img_fname)
            lab = nii_loader(ct_set, label_fname)


            img = pad3d(img.get_data())
            img = image_preprocess(img,new_size=256)

            lab = pad3d(lab.get_data())
            lab = image_preprocess(lab,new_size=256,mask=True)



            print("Start MASKING"+"====="*10)

            List = {'filename':file_name,'image': img, 'label':  lab}
            pickler.dump(List)

            del file_name, img_fname, label_fname, img, lab
            
            print("Dumping",cnt)
            cnt += 1

        except MemoryError:
            print("Memory Error")
            todolist = total_list[cnt:]
            break

outpath = outpath[:-4] + str(int(outpath[-5]) + 1) + str(".pkl")




  0%|                                                                                           | 0/11 [00:00<?, ?it/s]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 9





  9%|███████▌                                                                           | 1/11 [00:33<05:36, 33.69s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 10





 18%|███████████████                                                                    | 2/11 [01:09<05:09, 34.40s/it]

...




 73%|████████████████████████████████████████████████████████████▎                      | 8/11 [05:03<01:57, 39.21s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 17





 82%|███████████████████████████████████████████████████████████████████▉               | 9/11 [05:36<01:14, 37.45s/it]

Memory Error


ValueError: invalid literal for int() with base 10: '.'

In [67]:
with open(outpath,"wb") as f:
    pickler = pickle.Pickler(f)
    for  record in tqdm(todolist):

        try :

            file_name = record['name']
            img_fname = record['image']
            label_fname = record['label']



            # nii load // 이 결과가가 원래 결과랑 같은 지 확인 하자
            img = nii_loader(ct_set,img_fname)
            lab = nii_loader(ct_set, label_fname)


            img = pad3d(img.get_data())
            img = image_preprocess(img,new_size=256)

            lab = pad3d(lab.get_data())
            lab = image_preprocess(lab,new_size=256,mask=True)



            print("Start MASKING"+"====="*10)

            List = {'filename':file_name,'image': img, 'label':  lab}
            pickler.dump(List)

            del file_name, img_fname, label_fname, img, lab
            
            print("Dumping",cnt)
            cnt += 1

        except MemoryError:
            print("Memory Error")
            todolist = total_list[cnt:]
            break

outpath = outpath[:-4] + str(int(outpath[-4]) + 1) + str(".pkl")





  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 18






 50%|██████████████████████████████████████████                                          | 1/2 [00:33<00:33, 33.28s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 19






100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:14<00:00, 35.60s/it]





In [78]:
pkl_itr1 = pickle_iter(r'D:\segmentiation\scipy_pickle_test0.pkl')
pkl_itr2 = pickle_iter(r'D:\segmentiation\scipy_pickle_test1.pkl')
pkl_itr3 = pickle_iter(r'D:\segmentiation\scipy_pickle_test2.pkl')

pkl_itr_list = [pkl_itr1, pkl_itr2, pkl_itr3]

tfrecords_filename = r"D:\segmentiation\scipy_tfrecord_compression.tfrecord"
write_image_annotation_pairs_to_tfrecord_from_listitr(pkl_itr_list, tfrecords_filename)


0
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
1
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
2
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
3
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
4
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
5
dict_keys(['filename', 'image', 'label'])
(256, 